In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from delta.tables import DeltaTable
from pyspark.sql.functions import col, desc
import requests
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType


StatementMeta(, 99ed4812-1864-49a3-b737-fedb0d38cff7, 4, Finished, Available, Finished)

In [35]:
spark = SparkSession.builder.getOrCreate()

StatementMeta(, 99ed4812-1864-49a3-b737-fedb0d38cff7, 37, Finished, Available, Finished)

In [37]:
def latest_date():
    delta_table = DeltaTable.forName(spark, "Bronze.dbo.taxi_config")
    df = delta_table.toDF()
    latest = df.orderBy(col("year").desc(), col("month").desc()).limit(2)

    latest.select(col('taxi_type'),col('year'),col('month'))
    rows = latest.select('taxi_type', 'year', 'month').collect()
    latest = {row['taxi_type']: {'year': row['year'], 'month': row['month']} for row in rows}
    return latest
def check_data_exist(latest):
    url="https://d37ci6vzurychx.cloudfront.net/trip-data/{}_tripdata_{}-{}.parquet"
    row=[]

    for taxi_type in latest.keys():
        year = int(latest[taxi_type]['year'])
        month = int(latest[taxi_type]['month'])
        if month == 12:
            month = 1
            year += 1
        else:
            month += 1

        month_str = str(month).zfill(2)
        response = requests.get(url.format(taxi_type, year, month_str))
        if response.status_code == 200:
            row.append((taxi_type, year, month_str, False))
    return row


StatementMeta(, 99ed4812-1864-49a3-b737-fedb0d38cff7, 39, Finished, Available, Finished)

yellow
green


In [ ]:
schema = StructType([
    StructField("taxi_type", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("ingested", BooleanType(), True),
])
latest=latest_date()
row=check_data_exist(latest)
df = spark.createDataFrame(row, schema)
df.write.format("delta").mode("append").saveAsTable("bronze.dbo.taxi_config")